## imoprts

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense ,GRU
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

## Preprocess Data and Create labels and features

In [ ]:
DATA_PATH=os.path.join('MP_Data')
actions = np.array(['pain','feaver','doctor','injury','yestrday','tomorow','injection','dizziness','allergy','bandage'])
no_sequences=30
sequence_length=30


In [ ]:
label_map={label:num for num,label in enumerate(actions)}

In [ ]:
sequences ,labels =[],[]
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res=np.load(os.path.join(DATA_PATH,action,str(sequence),f'{frame_num}.npy'))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
X=np.array(sequences)

In [ ]:
y= to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

## Build and train LSTM Neural Network

In [ ]:
log_dir=os.path.join("Logs")
tb_callback=TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["categorical_accuracy"])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

## Model prediction

In [ ]:
res=model.predict(X_test)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

## Save Weights

In [ ]:
model.save('actionv2.h5')

## Evaluation using Confusion Matrix and Accuracy

In [ ]:
yhat=model.predict(X_test)

In [ ]:
ytrue=np.argmax(y_test,axis=1).tolist()
yhat=np.argmax(yhat,axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue,yhat)

In [ ]:
accuracy_score(ytrue,yhat)

 ## Test in Real time

In [ ]:
model.load_weights('actionv2.h5')

In [ ]:
sequence=[]
sentence=[]
predictions=[]
threshold=0.7

cap=cv2.VideoCapture(0)
 
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        image,results=mediapipe_detection(frame,holistic)
        #print(results)
        draw_styled_landmarks(image,results)
        
        keypoints=extract_keypoints(results)
        sequence.append(keypoints)
        sequence=sequence[-30:]
        
        if len(sequence)==30:
            res=model.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)]>threshold:
                    if len(sentence)>0:
                        if actions[np.argmax(res)]!=sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else: 
                        sentence.append(actions[np.argmax(res)])

            if len(sentence)>5:
                sentence=sentence[-5:]
        
        cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
        cv2.putText(image,' '.join(sentence),(100,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        cv2.namedWindow("OpenCV Feed", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("OpenCV Feed", 1000, 800)
        cv2.imshow('OpenCV Feed',image)
        if cv2.waitKey(10) & 0xff==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()